# Alpaca Instuction Tuning Evalutation
Name: Sitthiwat Damrongpreechar <br>
StudentID: st123994

In [40]:
# install libraries
# !pip3 install peft==0.7.1
# !pip3 install trl==0.7.4
# !pip3 install transformer==4.36.2

In [41]:
import transformers, trl
transformers.__version__,trl.__version__

('4.36.2', '0.7.4')

In [42]:
# setting up the device
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

## 1. Load Dataset

In [43]:
# load the dataset "alpaca_data.json"
from datasets import load_dataset
train_dataset = load_dataset('json', data_files='./alpaca_data.json', split='train')
eval_dataset = load_dataset("tatsu-lab/alpaca_eval", split='eval', trust_remote_code=True)
eval_dataset = eval_dataset.remove_columns(["generator", "dataset"])

FileNotFoundError: Unable to find 'C:/Users/earth/Desktop/NLP/A8_Instruction_Tuning_with_hf-AlpacaEval\./alpaca_data.json'

In [ ]:
train_dataset

Dataset({
    features: ['instruction', 'output', 'input'],
    num_rows: 52002
})

In [ ]:
eval_dataset

Dataset({
    features: ['instruction', 'output'],
    num_rows: 805
})

## 2. Load the model and Tokenizer

In [44]:
from transformers import AutoModelForCausalLM, AutoTokenizer
model_name_or_path = "distilgpt2"
model = AutoModelForCausalLM.from_pretrained(
    model_name_or_path, device_map = 'auto')
tokenizer = AutoTokenizer.from_pretrained(
    model_name_or_path)
tokenizer.pad_token = tokenizer.eos_token
# Make sure to pass a correct value for max_seq_length as the default value will be set to min(tokenizer.model_max_length, 1024).
max_seq_length = min(tokenizer.model_max_length, 1024)
max_seq_length

1024

## 3. Prompts

In [45]:
def formatting_prompts_func(examples):
	output_texts = []

	for i in range(len(examples['instruction'])):
		if 'input' in examples.keys():
			input_text = examples["input"][i] 
		else:
			input_text = None
	
		if input_text:
			text = f"""
Below is an instruction for an instruction-tuning task, paired with an input that provides further context. Write a response that appropriately aligns with the provided instruction.

### Instruction:
{examples["instruction"][i]}

### Input:
{input_text}

### Response:
{examples["output"][i]}
""".strip()
			
		else:
			text = f"""
Below is an instruction for an instruction-tuning task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{examples["instruction"][i]}

### Response:
{examples["output"][i]}
""".strip()

		output_texts.append(text)

	return output_texts

In [46]:
# use the DataCollatorForCompletionOnlyLM to train your model on the generated prompts only
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM
response_template = "### Response:"
collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)
collator

DataCollatorForCompletionOnlyLM(tokenizer=GPT2TokenizerFast(name_or_path='distilgpt2', vocab_size=50257, model_max_length=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>', 'pad_token': '<|endoftext|>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	50256: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
}, mlm=False, mlm_probability=0.15, pad_to_multiple_of=None, tf_experimental_compile=False, return_tensors='pt')

## 4. Train the model

In [47]:
from transformers import TrainingArguments
# path to save the model
path = './checkpoints'
# TrainingArguments
training_args = TrainingArguments(
    output_dir = path, #default = 'tmp_trainer'
    save_strategy = 'epoch',
    gradient_checkpointing = True,
    per_device_train_batch_size = 2,
    per_device_eval_batch_size = 2,
    num_train_epochs = 5, #default = 3
)

In [48]:
# instantiate the SFTTrainer
trainer = SFTTrainer(
    model,
    args = training_args,
    train_dataset = train_dataset.select(range(10000)),
    eval_dataset = eval_dataset,
    formatting_func = formatting_prompts_func,
    data_collator = collator,
    max_seq_length = max_seq_length,
)


In [18]:
# train the model
trainer.train()

  0%|          | 0/25000 [00:00<?, ?it/s]

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
c:\Users\earth\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\utils\checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


{'loss': 2.6757, 'learning_rate': 4.9e-05, 'epoch': 0.1}
{'loss': 2.6376, 'learning_rate': 4.8e-05, 'epoch': 0.2}
{'loss': 2.5541, 'learning_rate': 4.7e-05, 'epoch': 0.3}
{'loss': 2.5301, 'learning_rate': 4.600000000000001e-05, 'epoch': 0.4}
{'loss': 2.4877, 'learning_rate': 4.5e-05, 'epoch': 0.5}
{'loss': 2.5148, 'learning_rate': 4.4000000000000006e-05, 'epoch': 0.6}
{'loss': 2.4772, 'learning_rate': 4.3e-05, 'epoch': 0.7}
{'loss': 2.4441, 'learning_rate': 4.2e-05, 'epoch': 0.8}
{'loss': 2.4245, 'learning_rate': 4.1e-05, 'epoch': 0.9}


Checkpoint destination directory ./checkpoints\checkpoint-5000 already exists and is non-empty.Saving will proceed but saved results may be invalid.


{'loss': 2.4522, 'learning_rate': 4e-05, 'epoch': 1.0}


...

Checkpoint destination directory ./checkpoints\checkpoint-20000 already exists and is non-empty.Saving will proceed but saved results may be invalid.


{'loss': 1.7427, 'learning_rate': 1e-05, 'epoch': 4.0}


c:\Users\earth\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\utils\checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


{'loss': 1.6179, 'learning_rate': 9e-06, 'epoch': 4.1}
{'loss': 1.5883, 'learning_rate': 8.000000000000001e-06, 'epoch': 4.2}
{'loss': 1.6334, 'learning_rate': 7.000000000000001e-06, 'epoch': 4.3}
{'loss': 1.6035, 'learning_rate': 6e-06, 'epoch': 4.4}
{'loss': 1.5947, 'learning_rate': 5e-06, 'epoch': 4.5}
{'loss': 1.6386, 'learning_rate': 4.000000000000001e-06, 'epoch': 4.6}
{'loss': 1.6409, 'learning_rate': 3e-06, 'epoch': 4.7}
{'loss': 1.5751, 'learning_rate': 2.0000000000000003e-06, 'epoch': 4.8}
{'loss': 1.5951, 'learning_rate': 1.0000000000000002e-06, 'epoch': 4.9}
{'loss': 1.6271, 'learning_rate': 0.0, 'epoch': 5.0}
{'train_runtime': 2296.3844, 'train_samples_per_second': 21.773, 'train_steps_per_second': 10.887, 'train_loss': 1.9754472119140625, 'epoch': 5.0}


TrainOutput(global_step=25000, training_loss=1.9754472119140625, metrics={'train_runtime': 2296.3844, 'train_samples_per_second': 21.773, 'train_steps_per_second': 10.887, 'train_loss': 1.9754472119140625, 'epoch': 5.0})

In [19]:
trainer.save_model('./app/model')

## 5. Inference

In [71]:
from transformers import pipeline
# load the model that we just trained
model = AutoModelForCausalLM.from_pretrained(
    './app/model',
    device_map = 'auto')
# create a text generation pipeline
text_generator = pipeline(
    "text-generation",
    model = model,
    tokenizer = tokenizer,
    pad_token_id = tokenizer.eos_token_id,
    max_new_tokens = 100,
    device_map = 'auto',
)

In [72]:
# generate prompt for the model
def Generate_prompt_input(text):
	
	if 'input' in text.keys():
		return f"""
Below is an instruction for an instruction-tuning task, paired with an input that provides further context. Write a response that appropriately aligns with the provided instruction.

### Instruction:
{text['instruction']}

### Input:
{text['input']}

### Response:
""".strip()
			
	else:
		return f"""
Below is an instruction for an instruction-tuning task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{text['instruction']}

### Response:
""".strip()

## 6. Comparing the generated result with gold label

In [78]:
result = text_generator(Generate_prompt_input(eval_dataset[10]))
print(result[0]['generated_text'],"\n")
print(f"### GOLD LABEL:\n{eval_dataset['output'][10]}") #gold label

Below is an instruction for an instruction-tuning task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
do you think retinoid is effective on removing the acne? because I have a lot of it

### Response:
Yes, retinoid can be effective on removing acne. Just like with regular acne treatments, it can remove any acne products, including streaks of red, pinks, and scabs. Plus, it can do anorectal removal to remove the spots where it needs to be removed. It's also great to get the necessary vitamins, including vitamins, minerals, and minerals to help remove spots or issues. All in all, retinoid is a good option if you need it the 

### GOLD LABEL:
Yes, retinoids are effective in treating acne. They work by increasing cell turnover, which helps to reduce the appearance of existing acne and prevent new outbreaks. Retinoids also help to unclog pores, which in turn reduces the amount of bacteria that can cause infec